In [ ]:
import tensorflow as tf
from tensorflow.python.ops import control_flow_ops
import numpy as np
import os
from tqdm import tqdm
from matplotlib import pyplot as plt
from skimage import io
from skimage import img_as_ubyte
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

DATASET_PATH = "datasets"
DATASET_TRAIN_PATH = "train"
DATASET_TEST_PATH = "test"
DATASET_OUTPUT_PATH = "outputs"

DEVSET_TOTAL_RATIO = 0.1

NUM_THREADS = 8

IMAGE_SIZE = 224

TRAINSET_SOURCE_PATH =  "%s/%s/%s" % (os.getcwd(), DATASET_PATH, DATASET_TRAIN_PATH)
TESTSET_SOURCE_PATH = "%s/%s/%s" % (os.getcwd(), DATASET_PATH, DATASET_TEST_PATH)

train_image_names = [i for i in os.listdir(TRAINSET_SOURCE_PATH)]
train_image_labels = np.array([(1 if 'dog' in i else 0) for i in train_image_names]).astype(np.int64)

DEVSET_TOTAL = int(len(train_image_names) * 0.1)
dev_images_idx = np.random.choice(len(train_image_names), DEVSET_TOTAL, replace=False)

full_train_image_path = ["%s/%s" % (TRAINSET_SOURCE_PATH, i) for i in train_image_names]

dev_images = [full_train_image_path[i] for i in dev_images_idx]
dev_labels = train_image_labels[dev_images_idx]

train_images = [elem for i, elem in enumerate(full_train_image_path) if i not in dev_images_idx]
train_labels = np.delete(train_image_labels, dev_images_idx)

test_images = ["%s/%s" % (TESTSET_SOURCE_PATH, i) for i in os.listdir(TESTSET_SOURCE_PATH)]
test_labels = (np.ones((len(test_images))) * -1).astype(np.int64)

print('Total Image Size: ', len(train_image_names))
print('Training Set Images Size: ', len(train_images))
print('Training Set Labels Shape: ', train_labels.shape)
print('Dev Set Images Size: ', len(dev_images))
print('Dev Set Labels Shape: ', dev_labels.shape)
print('Test Set Images Size:', len(test_images))

In [ ]:
def resize_padded(img, size):   
    from skimage.transform import resize
    
    width = img.shape[0]
    height = img.shape[1]
    channel = img.shape[2]
    new_width = width
    new_height = height
    x = 0
    y = 0
    if height > width:
        new_height = size
        new_width = int((width/height) * size)
        y = 0
        x = int(( size - new_width ) / 2)
    else:        
        new_width = size
        new_height = int((height/width) * size)
        x = 0
        y = int(( size - new_height ) / 2)
    
    interm_shape = (new_width, new_height)
    new_shape = (size, size, channel)
    
    interm_img = resize(img, interm_shape)
    
    new_img = np.empty((size, size, channel), dtype=interm_img.dtype)
    new_img.fill(0)
    
    new_img[x:new_width+x, y:new_height+y] = interm_img
    
    return img_as_ubyte(new_img)


In [ ]:
image_examples_paths = [train_images[i] for i in np.random.choice(len(train_images), 3, replace=False)]
plt.figure()
plt.axis('off')
for i, file_path in enumerate(image_examples_paths):
    plt.subplot(2,3,i + 1)
    image = io.imread(file_path)
    img = plt.imshow(image, interpolation='nearest')
    img.axes.get_xaxis().set_visible(False)
    img.axes.get_yaxis().set_visible(False)

    image_resized = resize_padded(image, IMAGE_SIZE)
    plt.subplot(2,3,i + 4)
    img = plt.imshow(image_resized, interpolation='nearest')
    img.axes.get_xaxis().set_visible(False)
    img.axes.get_yaxis().set_visible(False)

In [ ]:
from datetime import datetime
import os
import sys
import threading

def _int64_feature(value):
    """Wrapper for inserting int64 features into Example proto."""
    if not isinstance(value, list):
        value = [value]
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))


def _bytes_feature(value):
    """Wrapper for inserting bytes features into Example proto."""
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def _convert_to_example(filename, image_buffer, label, width, height):
    """Build an Example proto for an example.
    Args:
    filename: string, path to an image file, e.g., '/path/to/example.JPG'
    image_buffer: string, JPEG encoding of RGB image
    label: integer, identifier for the ground truth for the network
    height: integer, image height in pixels
    width: integer, image width in pixels
    Returns:
    Example proto
    """
    example = tf.train.Example(features=tf.train.Features(feature={
        'width': _int64_feature(width),
        'height': _int64_feature(height),
        'label': _int64_feature(label),
        'filename': _bytes_feature(tf.compat.as_bytes(os.path.basename(filename))),
        'image': _bytes_feature(tf.compat.as_bytes((image_buffer)))
    }))
    return example


def _process_image(filename):
    """Process a single image file.
    Args:
      filename: string, path to an image file e.g., '/path/to/example.JPG'.
    Returns:
      image_buffer: string, JPEG encoding of RGB image.
      height: integer, image height in pixels.
      width: integer, image width in pixels.
    """
    image = resize_padded(io.imread(filename), IMAGE_SIZE)

    # Check that image converted to RGB
    assert len(image.shape) == 3
    height = image.shape[0]
    width = image.shape[1]
    assert image.shape[2] == 3

    return image.tobytes(), height, width

def _process_image_files_batch(thread_index, ranges, name, filenames,
                               labels, num_shards):
    """Processes and saves list of images as TFRecord in 1 thread.
    Args:
      thread_index: integer, unique batch to run index is within [0, len(ranges)).
      ranges: list of pairs of integers specifying ranges of each batches to
        analyze in parallel.
      name: string, unique identifier specifying the data set
      filenames: list of strings; each string is a path to an image file
      labels: list of integer; each integer identifies the ground truth
      num_shards: integer number of shards for this data set.
    """
    # Each thread produces N shards where N = int(num_shards / num_threads).
    # For instance, if num_shards = 128, and the num_threads = 2, then the first
    # thread would produce shards [0, 64).
    num_threads = len(ranges)
    assert not num_shards % num_threads
    num_shards_per_batch = int(num_shards / num_threads)

    shard_ranges = np.linspace(ranges[thread_index][0],
                               ranges[thread_index][1],
                               num_shards_per_batch + 1).astype(int)
    num_files_in_thread = ranges[thread_index][1] - ranges[thread_index][0]

    counter = 0
    for s in range(num_shards_per_batch):
        # Generate a sharded version of the file name, e.g. 'train-00002-of-00010'
        shard = thread_index * num_shards_per_batch + s
        output_filename = '%s-%.5d-of-%.5d.tfrecord' % (name, shard, num_shards)
        output_file = os.path.join(DATASET_OUTPUT_PATH, output_filename)
        writer = tf.python_io.TFRecordWriter(output_file)

        shard_counter = 0
        files_in_shard = np.arange(shard_ranges[s], shard_ranges[s + 1], dtype=int)
        for i in files_in_shard:
            filename = filenames[i]
            label = labels[i]

            image_buffer, height, width = _process_image(filename)

            example = _convert_to_example(filename, image_buffer, label, width, height)
            writer.write(example.SerializeToString())
            shard_counter += 1
            counter += 1

            if not counter % 1000:
                print('%s [thread %d]: Processed %d of %d images in thread batch.' %
                      (datetime.now(), thread_index, counter, num_files_in_thread))
                sys.stdout.flush()

        writer.close()
        print('%s [thread %d]: Wrote %d images to %s' %
              (datetime.now(), thread_index, shard_counter, output_file))
        sys.stdout.flush()
        shard_counter = 0
        print('%s [thread %d]: Wrote %d images to %d shards.' %
              (datetime.now(), thread_index, counter, num_files_in_thread))
        sys.stdout.flush()


def process_image_files(name, filenames, labels, num_shards):
    """Process and save list of images as TFRecord of Example protos.
    Args:
    name: string, unique identifier specifying the data set
    filenames: list of strings; each string is a path to an image file
    labels: list of integer; each integer identifies the ground truth
    num_shards: integer number of shards for this data set.
    """
    assert len(filenames) == len(labels)

    # Break all images into batches with a [ranges[i][0], ranges[i][1]].
    spacing = np.linspace(0, len(filenames), NUM_THREADS + 1).astype(np.int)
    ranges = []
    for i in range(len(spacing) - 1):
        ranges.append([spacing[i], spacing[i + 1]])

    # Launch a thread for each batch.
    print('Launching %d threads for spacings: %s' % (NUM_THREADS, ranges))
    sys.stdout.flush()

    # Create a mechanism for monitoring when all threads are finished.
    coord = tf.train.Coordinator()


    threads = []
    for thread_index in range(len(ranges)):
        args = (thread_index, ranges, name, filenames,
                labels, num_shards)
        t = threading.Thread(target=_process_image_files_batch, args=args)
        t.start()
        threads.append(t)

    # Wait for all the threads to terminate.
    coord.join(threads)
    print('%s: Finished writing all %d images in data set.' %
          (datetime.now(), len(filenames)))
    sys.stdout.flush()

In [ ]:
process_image_files('dev', dev_images, dev_labels, 8)

In [ ]:
process_image_files('test', test_images, test_labels, 8)

In [ ]:
process_image_files('train', train_images, train_labels, 8)